In [2]:
import os
import pydicom
from pathlib import Path
from monai.data import Dataset, DataLoader
import monai.transforms as transforms
import matplotlib as plt

In [41]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
#from monai.handlers.utils import from_engine

import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
print_config()


MONAI version: 0.8.1
Numpy version: 1.19.5
Pytorch version: 1.10.1+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 71ff399a3ea07aef667b23653620a290364095b1

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.2
scikit-image version: 0.13.1
Pillow version: 8.4.0
Tensorboard version: 2.10.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.11.2+cu102
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.4.5
pandas version: 0.23.0
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [175]:

# Parent directory where "train" and "test" folders are located
parent_directory = '//mnt/c/Users/ahmad/OneDrive - Delft University of Technology/studi/TUD/Internship/Clean_data/'

# Function to extract label from the folder path
def extract_label(folder_path):
    # Split the folder path and take the last two elements as the label
    components = folder_path.split(os.path.sep)
    label = os.path.join(components[-1])
    return label

# Create a list of dictionaries with keys "path" and "label"
train_list = []
test_list = []

# Iterate through "train" and "test" folders

for split in ["Train Sample", "Test Sample"]:
    split_folder = os.path.join(parent_directory, split)
    # Iterate through DICOM folders within the split folder
    if split == "Train Sample":
        for root, _, files in os.walk(split_folder):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_file_path = os.path.join(root, file)
                    label = extract_label(root)
                    train_list.append({"path": dicom_file_path, "label": label})
            print(f"Slice number: {len(files)}")
    else:
        for root, _, files in os.walk(split_folder):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_file_path = os.path.join(root, file)
                    label = extract_label(root)
                    test_list.append({"path": dicom_file_path, "label": label})
        


# Define a custom Dataset class to load DICOM files and extract data
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        super().__init__(data, transform)

    def __getitem__(self, index):
        dicom_file_path = self.data[index]["path"]
        label = self.data[index]["label"]

        # Load the DICOM file and extract information
        dicom_data = pydicom.dcmread(dicom_file_path)
        
        # DICOM header attributes
        repetition_time = int(float(dicom_data.RepetitionTime))
        echo_time = int(float(dicom_data.EchoTime))
        magnetic_strength = round(dicom_data.MagneticFieldStrength, 1)

        return {"path": dicom_file_path, "label": label, "metadata": {"repetition_time": repetition_time, "echo_time": echo_time, "magnetic_strength": magnetic_strength}}

# Create a MONAI dataset
train = CustomDataset(data=train_list)
test = CustomDataset(data=test_list)



Slice number: 32


In [133]:
train = []
test = []
for i in range (n_train):
    train.append(dataset[i])
for j in range(n_train, len(dataset)):
    test.append(dataset[j])


i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100
i: 101
i: 102
i: 103
i: 104
i: 105
i: 106
i: 107
i: 108
i: 109
i: 110
i: 111
i: 112
i: 113
i: 114
i: 115
i: 116
i: 117
i: 118
i: 119
i: 120
i: 121
i: 122
i: 123
i: 124
i: 125
i: 126
i: 127
i: 128
i: 129
i: 130
i: 131
i: 132
i: 133
i: 134
i: 135
i: 136
i: 137
i: 138
i: 139
i: 140
i: 141
i: 142
i: 143
i: 144
i: 145
i: 146
i: 147
i: 148
i: 149
i: 150
i: 151
i: 152
i: 153
i: 154
i: 155
i: 156
i: 157
i: 1

In [172]:
train[25]['metadata']

{'repetition_time': 185, 'echo_time': 7, 'magnetic_strength': 1.0}

In [61]:
for batch in data_loader:
    batch_data = batch[0]
    dicom_file_path = batch_data["path"]
    label = batch_data["label"]
    repetition_time = batch_data["repetition_time"]
    echo_time = batch_data["echo_time"]

    print(f"DICOM file path: {dicom_file_path}")
    print(f"Label: {label}")
    print(f"Repetition Time: {repetition_time}")
    print(f"Echo Time: {echo_time}")
    print()

KeyError: 0

In [39]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
    ]
)

In [40]:
check_ds = Dataset(data=val_files, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
image, label = (check_data["image"][0][0], check_data["label"][0][0])
print(f"image shape: {image.shape}, label shape: {label.shape}")
# plot the slice [:, :, 80]
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, 80], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label[:, :, 80])
plt.show()

NameError: name 'val_files' is not defined